# Exercise: One-hot vector

[One-hot encoding](https://en.wikipedia.org/wiki/One-hot) is useful in machine learning.

It simply means that among a group of bits (all either 0 or 1) only one is hot (1) while all others are cold (0),

`v = [0, 0, 0, 0, 0, 1, 0, 0, 0]`

### Task

1. Think about what information an implementation of a one-hot vector actually has to store.
2. Define a `OneHot` type which represents a vector with only a single hot (i.e. `== 1`) bit.
3. Extend all the necessary `Base` functions such that the following computation works for a matrix `A` and a vector of `OneHot` vectors `vs` (i.e. `vs isa Vector{OneHot}`).

    ```julia
    function innersum(A, vs)
        t = zero(eltype(A))
        for v in vs
            y = A*v
            for i in 1:length(vs[1])
                t += v[i] * y[i]
            end
        end
        return t
    end

    A = rand(3,3)
    vs = [rand(3) for i in 1:10] # This should be replaced by a `Vector{OneHot}`

    innersum(A, vs)

    ```

4. Benchmark the speed of `innersum` when called with a vector of `OneHot` vectors (i.e. `vs = [OneHot(3, rand(1:3)) for i in 1:10]`) and when called with a vector of `Vector{Float64}` vectors, respectively.
 * Do you observe a speed up?


5. Now, define a `OneHotVector` type which is identical to `OneHot` but is declared to be a subtype of `AbstractVector{Bool}` and extend only the functions `Base.getindex(v::OneHotVector, i::Int)` and `Base.size(v::OneHotVector)`.
 * Here, the function `size` should return a `Tuple{Int64}` indicating the length of the vector, i.e. `(3,)` for a one-hot vector of length 3.
 

6. Try to create a single `OneHotVector` and try to run the `innersum` function using the new `OneHotVector` type.
 * What changes do you observe?
 * Do you have to implement any further methods?

In [1]:
import Base: *, getindex, length

In [2]:
struct OneHot
    len::Int64
    hot::Int64
    OneHot(l, i) = l < i ? error("Length must be longer than position of 1") : new(l, i)
end

In [3]:
*(M::Matrix{T}, V::OneHot) where T<:Real = M[:, V.hot]
#*(a::Array{T}, V::OneHot) where T<:Real = a[V.hot]
Base.length(V::OneHot) = V.len
Base.getindex(V::OneHot, i::Int) = Int(i == V.hot)

In [4]:
v = OneHot(10, 1)

OneHot(10, 1)

In [5]:
v[5]

0

In [6]:
M = rand(2,2)
M[:, 1]

2-element Vector{Float64}:
 0.41486929754296265
 0.4873007317510589

In [7]:
M * v

2-element Vector{Float64}:
 0.41486929754296265
 0.4873007317510589

In [10]:
length(v)
length(vs[1])

LoadError: UndefVarError: vs not defined

In [11]:
function innersum(A, vs)
     t = zero(eltype(A))
     for v in vs
         y = A*v
         for i in 1:length(vs[1])
             t += v[i] * y[i]
         end
     end
     return t
 end

innersum (generic function with 1 method)

In [20]:
A = rand(3,3)
vs = [OneHot(3, rand(1:3)) for _ in 1:10] # This should be replaced by a `Vector{OneHot}`

10-element Vector{OneHot}:
 OneHot(3, 2)
 OneHot(3, 3)
 OneHot(3, 2)
 OneHot(3, 2)
 OneHot(3, 2)
 OneHot(3, 1)
 OneHot(3, 2)
 OneHot(3, 2)
 OneHot(3, 1)
 OneHot(3, 3)

In [15]:
innersum(A, vs)

4.5185399324615085

In [17]:
struct OneHotVector <: AbstractVector{Bool}
    len::Int
    hot::Int    
    OneHotVector(l, i) = l < i ? error("Length must be longer than position of 1") : new(l, i)
end 

In [18]:
Base.getindex(V::OneHotVector, i::Int) = Int(V.hot==i)
Base.size(V::OneHotVector) = V.len

In [21]:
A = rand(3,3)
vs2 = [OneHotVector(3, rand(1:3)) for _ in 1:10]

innersum(A, vs)

2.473392534095634

In [23]:
vs_float64 = [rand(3) for _ in 1:10]
using BenchmarkTools

@btime innersum($A, $vs);
@btime innersum($A, $vs2)
@btime innersum($A, $vs_float64);

  366.471 ns (10 allocations: 800 bytes)
  441.934 ns (10 allocations: 800 bytes)
  675.032 ns (10 allocations: 800 bytes)
